# Task 1

Original submission file format:

```
{'target': {'query_1': {'partnumber': 17265},
  'query_2': {'user_id': 34572},
  'query_3': {'average_previous_visits': 5.52},
  'query_4': {'device_type': 23},
  'query_5': {'user_id': 123734},
  'query_6': {'unique_families': 2357},
  'query_7': {'1': 3, '2': 5, '3': 3, '4': 9, '5': 5, '6': 1}}}
```

In [1]:
import os
import sys

# root path
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if ROOT not in sys.path:
    sys.path.append(ROOT)

import polars as pl
import pandas as pd
from src.data.loaders import PolarsLoader

## Paths

In [2]:
from config import (USERS_DATA_PATH, TRAIN_DATA_PATH, 
                    TEST_DATA_PATH, SUBMISSION_1_PATH, EX_SUBMISSION_1_PATH, 
                    PRODUCTS_PARQUET_PATH, PRODUCTS_DATA_PATH,
                    TRAIN_PARQUET_PATH, TEST_PARQUET_PATH)

In [73]:
# Load sample submission json
import json
submission = json.load(open(EX_SUBMISSION_1_PATH))

## Query 1

**Q1:** Which product (`partnumber`) with `color_id` equal to 3   belongs to the lowest `familiy` code with a `discount`? 

In [74]:
# prods = pl.from_pandas(pd.read_pickle(PRODUCTS_DATA_PATH))

prod_loader = PolarsLoader(sampling=False, file_type='parquet')
prods = prod_loader.load_data(PRODUCTS_PARQUET_PATH)

q1 = prods.sql("""
          SELECT partnumber
          FROM self
          WHERE color_id = 3
          AND discount = 1
          AND family = MIN(family)
          """).item()

print(f"q1: {q1}")
submission['target']['query_1'] = {'partnumber': q1}

q1: 17265


## Query 2

**Q2:** In the country where most users have made purchases totaling less than 500 (`M`) , which is the user who has the lowest purchase frequency (`F`), the most recent purchase (highest `R`) and the lowest `user_id`? Follow the given order of variables as the sorting priority.

In [3]:
loader = PolarsLoader(sampling=False, file_type='parquet')
users = loader.load_data(USERS_DATA_PATH)

In [76]:
country = (users
           .filter(pl.col('M') < 500)
           .group_by('country')
           .len()
           .sort(by='len', descending=True)
           .head(1)
           )['country'].item()
country

25

In [77]:
q2 = (users
 .filter(pl.col('country') == country)
 .sort("F", "R", "user_id", descending=[False, True, False])
 .head(1)
)['user_id'].item()

print(f"Q2: {q2}")
submission['target']['query_2'] = {'user_id': q2}

Q2: 187374


## Query 3 (Run on Kaggle)

**Q3:** Among the products that were added to the cart at least once, how many times is a product visited before it is added to the cart in average? Give the answer with 2 decimals.

In [78]:
# %%time
# loader = PolarsLoader(sampling=True)

# # Load data lazily
# train = (loader
#          .load_data_lazy(path=TRAIN_DATA_PATH)
#          .select(["session_id", "partnumber", "add_to_cart", "timestamp_local"])
#         )

# # Products that were added to the cart
# products_added = (
#     train
#     .filter(pl.col("add_to_cart") == 1)
#     .select("partnumber")
#     .unique()  # Stay in LazyFrame
# )

# # Keep only interactions for these products
# interactions_for_cart_products = (
#     train
#     .join(products_added, on="partnumber", how="inner")  # Lazy join instead of is_in
#     .sort(["session_id", "partnumber", "timestamp_local"])  # Sorting will also be deferred
# )

# # Add a cumulative flag for add_to_cart in each group
# grouped_data = (
#     interactions_for_cart_products
#     .group_by(["session_id", "partnumber"])
#     .agg([
#         pl.col("add_to_cart").cum_sum().alias("add_to_cart_cumsum"),
#         pl.col("add_to_cart"),
#         pl.col("timestamp_local"),
#     ])
#     .explode(["add_to_cart_cumsum", "add_to_cart", "timestamp_local"]) 
# )

# # Calculate pre-cart visits
# pre_cart_visits = (
#     grouped_data
#     .filter(pl.col("add_to_cart_cumsum") == 0)
#     .group_by("partnumber")
#     .agg(pl.col("session_id").count().alias("visit_count"))
# )

# # Collect only the final aggregated result
# final_result = pre_cart_visits.collect(streaming=True)

# # Calculate the average visits before add_to_cart
# average_visits = round(final_result["visit_count"].mean(), 2)

# print("Average Visits Before Adding to Cart:", average_visits)

**Testing loading the new train data as parquet casted down**

In [79]:
%%time
loader = PolarsLoader(sampling=True, file_type="csv")
df = loader.load_data(TRAIN_DATA_PATH)
df.group_by("session_id").agg(pl.col("add_to_cart").sum().alias("add_to_cart")).head()

CPU times: user 3.67 s, sys: 252 ms, total: 3.92 s
Wall time: 677 ms


session_id,add_to_cart
i64,i64
2220794,1
1205834,0
1303879,0
2454904,0
4121196,0


In [80]:
%%time
loader = PolarsLoader(sampling=True, file_type="parquet")
df = loader.load_data(TRAIN_PARQUET_PATH)
df.group_by("session_id").agg(pl.col("add_to_cart").sum().alias("add_to_cart")).head()

CPU times: user 225 ms, sys: 29.6 ms, total: 254 ms
Wall time: 69.5 ms


session_id,add_to_cart
u32,i64
419598,0
4531085,0
4498409,1
4168505,1
4978720,0


In [81]:
average_visits = 1458.40 # From Kaggle run
submission['target']['query_3'] = {'average_previous_visits': average_visits}

## Query 4

**Q4:** Which device (`device_type`) is most frequently used by users to make purchases (`add_to_cart` = 1) of discounted products (`discount` = 1)?

In [82]:
SAMPLING = False

In [83]:
prod_loader = PolarsLoader(sampling=SAMPLING, file_type='parquet')
prods = prod_loader.load_data(PRODUCTS_PARQUET_PATH, select_cols=['discount', 'partnumber'])

prods = prods.filter(pl.col('discount') == 1)

In [84]:
loader = PolarsLoader(sampling=SAMPLING, file_type="parquet")
train = loader.load_data(TRAIN_PARQUET_PATH).filter((pl.col("add_to_cart") == 1))

In [85]:
filtered_data = train.join(prods, on='partnumber', how='inner')

In [86]:
# import random
# filtered_data.filter(pl.col("session_id") == random.choice(filtered_data["session_id"].unique())) # one device per session id

In [87]:
# Count device usage
device_counts = (
    filtered_data.group_by("device_type")
    .agg(pl.col("session_id").count().alias("count"))
)
device_counts

device_type,count
u8,u32
3,16416
1,169439
2,1884


In [88]:
# Find the most frequently used device
most_frequent_device = (
    device_counts
    .sort("count", descending=True)
    .select(pl.first("device_type"))
    .item()
)

print("Most Frequent Device for Discounted Purchases:", most_frequent_device)

Most Frequent Device for Discounted Purchases: 1


In [89]:
submission['target']['query_4'] = {'device_type': most_frequent_device}

## Query 5

**Q5:** Among users with purchase frequency (`F`) in the top 3 within their purchase country, who has interacted with the most products (`partnumber`) in sessions conducted from a device with identifierr 3 (`device_type` = 3)?

In [90]:
loader = PolarsLoader(sampling=False, file_type='parquet')
users = loader.load_data(USERS_DATA_PATH)
print(len(users))

# Rank users by F within each country
ranked_users = (users
                #     .filter(pl.col("country") == 25) ########## REMOVE
                    .group_by("country")
                    .agg([pl.col("user_id"),pl.col("F")])
                    .explode(["user_id","F"])
                    .sort(by=["country", "F"], descending=[False, True])
                    .with_columns(
                            pl.col("F").rank(method="ordinal", descending=True).over("country").alias("rank_in_country")
                        )
                )      

557006


In [91]:
# Filter top 3 users by F within each country
top_users = ranked_users.filter(pl.col("rank_in_country") <= 3)
top_users

country,user_id,F,rank_in_country
u8,u32,u16,u32
1,162396,7,1
1,65377,4,2
1,151995,2,3
3,158760,8,1
4,269481,355,1
…,…,…,…
86,35439,11,1
88,90168,1,1
89,259199,60,1


In [92]:
loader = PolarsLoader(sampling=False, file_type="parquet")
train = loader.load_data(TRAIN_PARQUET_PATH).filter(pl.col("device_type") == 3)

In [93]:
# Find the user with the most unique product interactions
top_user_interactions = (train
 .filter(pl.col("device_type") == 3)
 .join(top_users, on="user_id", how="inner")
)

unique_product_counts = (
    top_user_interactions.group_by("user_id")
    .agg([
        pl.col("partnumber").n_unique().alias("unique_products")
    ])
)

most_active_user = (
    unique_product_counts.sort("unique_products", descending=True)
    .select(pl.first("user_id"))
    .item()
)

most_active_user

9664

In [94]:
submission['target']['query_5'] = {'user_id': most_active_user}

## Query 6

**Q6:** For interactions that occurred outside the user's country of residence, how many unique family identifiers are there?

In [95]:
users = users.rename({"country": "user_country"})
loader = PolarsLoader(sampling=False, file_type="parquet")
train = loader.load_data(TRAIN_PARQUET_PATH)

In [96]:
# Filter interactions outside the user's country
outside_country_interactions = (train
 .join(users, on="user_id", how="left")
 .filter(pl.col("country") != pl.col("user_country"))
)

In [97]:
prod_loader = PolarsLoader(sampling=False, file_type='parquet')
prods = prod_loader.load_data(PRODUCTS_PARQUET_PATH, select_cols=['partnumber', 'family'])
prods.head()

partnumber,family
u16,u16
32776,73
41431,73
39419,73
36087,73
34132,73


In [98]:
# Count unique family identifiers
unique_families = outside_country_interactions.join(prods, on='partnumber', how='inner').select("family").n_unique()
unique_families

108

In [99]:
submission['target']['query_6'] = {'unique_families': unique_families}

## Query 7

**Q7:** Among interactions from the first 7 days of June, which is the most frequent page type where each family is added to the cart? Return it in the following format: `{'('family'): int('most_frequent_pagetype')}` . In case of a tie, return the smallest pagetype.

In [100]:
loader = PolarsLoader(sampling=False, file_type='parquet')

train = loader.load_data(TRAIN_PARQUET_PATH)

In [101]:
start_date = pd.to_datetime("2024-06-01")
end_date = pd.to_datetime("2024-06-07")


train = train.filter((pl.col("timestamp_local") >= start_date) & (pl.col("timestamp_local") <= end_date) & (pl.col("add_to_cart") == 1))

In [102]:
prods = loader.load_data(PRODUCTS_PARQUET_PATH, select_cols=['family', 'partnumber'])

In [103]:
# Join products data, group by family and pagetype, and count occurrences 
q7_data = train.join(prods, on='partnumber', how='left')

pagetype_counts = (q7_data.group_by(["family", "pagetype"])
                    .agg(pl.col("session_id").count().alias("count"))  # Count occurrences
                )

In [104]:
# Find the most frequent pagetype for each family
most_frequent_pagetype = (
    pagetype_counts.sort(["family", "count", "pagetype"], descending=[False, True, False])
    .group_by("family")
    .agg(pl.col("pagetype").first())  # Select the most frequent pagetype
)

In [105]:
q7_result = {row["family"]: row["pagetype"] for row in most_frequent_pagetype.to_dicts()}

q7_result

{1: 24,
 2: 24,
 3: 24,
 4: 24,
 5: 24,
 6: 24,
 7: 24,
 8: 24,
 9: 24,
 10: 24,
 11: 24,
 12: 24,
 13: 24,
 14: 24,
 15: 24,
 16: 24,
 17: 24,
 18: 24,
 19: 24,
 21: 24,
 22: 24,
 23: 24,
 24: 24,
 26: 24,
 27: 24,
 28: 24,
 29: 24,
 30: 24,
 31: 24,
 32: 24,
 33: 24,
 34: 24,
 35: 24,
 36: 24,
 37: 24,
 38: 24,
 40: 24,
 41: 24,
 42: 24,
 43: 24,
 44: 24,
 45: 24,
 46: 24,
 47: 24,
 48: 24,
 49: 24,
 50: 24,
 51: 24,
 52: 24,
 53: 24,
 54: 24,
 56: 24,
 57: 24,
 58: 24,
 59: 24,
 60: 24,
 61: 24,
 62: 24,
 63: 24,
 64: 24,
 65: 24,
 66: 24,
 67: 24,
 68: 24,
 69: 24,
 70: 24,
 71: 24,
 72: 24,
 73: 24,
 74: 24,
 75: 24,
 76: 24,
 77: 24,
 78: 24,
 79: 24,
 81: 24,
 82: 24,
 83: 24,
 84: 24,
 85: 24,
 86: 24,
 87: 24,
 88: 24,
 89: 24,
 90: 24,
 91: 24,
 92: 24,
 93: 24,
 94: 24,
 95: 24,
 96: 24,
 97: 24,
 99: 24,
 100: 24,
 101: 24,
 102: 24,
 103: 24,
 104: 24,
 105: 24,
 106: 24,
 107: 24,
 108: 24,
 109: 24,
 110: 24,
 111: 24,
 112: 24,
 113: 24,
 114: 24,
 115: 24,
 116: 24,
 1

In [106]:
submission['target']['query_7'] = q7_result

# Replace submission file

In [107]:
with open(SUBMISSION_1_PATH, 'w') as f:
    json.dump(submission, f, indent=4)